In [2]:
from timm.models import create_model
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from dataset import DrowsinessDataset
from torchvision import transforms

/home/ameenudeenp/miniconda/envs/work/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model=create_model("fastvit_t8")
torch.save({
    'state_dict': model.state_dict(),
    
}, 'fastvit_t8.pth.tar')


checkpoint = torch.load('/home/ameenudeenp/charu/ViT/fastvit_t8.pth.tar')
model.load_state_dict(checkpoint['state_dict'])


<All keys matched successfully>

In [4]:
print(model)

FastVit(
  (stem): Sequential(
    (0): MobileOneBlock(
      (se): Identity()
      (conv_kxk): ModuleList(
        (0): ConvNormAct(
          (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): Identity()
          )
        )
      )
      (conv_scale): ConvNormAct(
        (conv): Conv2d(3, 48, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (bn): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
      )
      (act): GELU(approximate='none')
    )
    (1): MobileOneBlock(
      (se): Identity()
      (conv_kxk): ModuleList(
        (0): ConvNormAct(
          (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48, bias=False)
          (bn): Batch

In [5]:
num_classes = 2 
model.head = torch.nn.Linear(model.head.in_features, num_classes)

In [14]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
train=pd.read_csv('/home/ameenudeenp/charu/ViT/train_annotations.csv')
train_dataset=DrowsinessDataset(train,transform=transform)
test=pd.read_csv('/home/ameenudeenp/charu/ViT/test_annotations.csv')
test_dataset=DrowsinessDataset(test,transform=transform)
val=pd.read_csv('/home/ameenudeenp/charu/ViT/val_annotations.csv')
val_dataset=DrowsinessDataset(val,transform=transform)

In [15]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_dl = DataLoader(train_dataset, batch_size=8)
val_dl = DataLoader(val_dataset,  batch_size=8)
test_dl = DataLoader(test_dataset, batch_size=8)

Using device: cpu


In [16]:
next(iter(train_dl))

/home/ameenudeenp/charu/ViT/DDD/Drowsy/X1523.png
0.1520727626436634
1
/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/v0815.png
0.2479285322112641
0
/home/ameenudeenp/charu/ViT/DDD/Drowsy/W1084.png
0.2846189758825745
1
/home/ameenudeenp/charu/ViT/DDD/Drowsy/A0098.png
0.226215483185079
1
/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/d0252.png
0.1913726478746589
0
/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/d0881.png
0.2689602560722339
0
/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/v0573.png
0.2470782866264517
0
/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/m0105.png
0.1737006830966679
0


{'pixel_values': tensor([[[[ 2.0434,  2.0434,  2.0263,  ..., -0.7650, -0.5424, -0.4226],
           [ 2.0434,  2.0263,  2.0263,  ..., -0.7822, -0.4568, -0.2342],
           [ 2.0263,  2.0263,  2.0092,  ..., -0.8164, -0.5082, -0.2856],
           ...,
           [-1.2788, -1.2959, -1.3302,  ..., -0.5596, -0.4568, -0.4397],
           [-1.3130, -1.3302, -1.3473,  ..., -0.6281, -0.4911, -0.4226],
           [-1.3473, -1.3644, -1.3473,  ..., -0.6965, -0.5596, -0.4568]],
 
          [[ 1.9559,  1.9384,  1.9209,  ..., -1.0028, -0.7752, -0.6176],
           [ 1.9559,  1.9209,  1.9209,  ..., -1.0203, -0.6702, -0.4251],
           [ 1.9209,  1.9209,  1.9034,  ..., -1.0553, -0.7402, -0.4951],
           ...,
           [-1.6856, -1.7031, -1.7206,  ..., -1.1604, -1.0378, -1.0378],
           [-1.7031, -1.7031, -1.7206,  ..., -1.2479, -1.1078, -1.0378],
           [-1.6856, -1.7031, -1.6856,  ..., -1.3354, -1.2304, -1.1253]],
 
          [[ 1.7337,  1.7337,  1.7163,  ..., -1.1421, -0.9330, -0.8110

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) 
criterion = torch.nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [10]:
model.train()

FastVit(
  (stem): Sequential(
    (0): MobileOneBlock(
      (se): Identity()
      (conv_kxk): ModuleList(
        (0): ConvNormAct(
          (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): Identity()
          )
        )
      )
      (conv_scale): ConvNormAct(
        (conv): Conv2d(3, 48, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (bn): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
      )
      (act): GELU(approximate='none')
    )
    (1): MobileOneBlock(
      (se): Identity()
      (conv_kxk): ModuleList(
        (0): ConvNormAct(
          (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48, bias=False)
          (bn): Batch

In [44]:
import torch.nn as nn

class FastViTModel(nn.Module):
    def __init__(self):
        super(FastViTModel,self).__init__()
        self.fastvit=create_model("fastvit_t8")
        torch.save({
            'state_dict': self.fastvit.state_dict(),
            
        }, 'fastvit_t8.pth.tar')
        checkpoint = torch.load('/home/ameenudeenp/charu/ViT/fastvit_t8.pth.tar')
        self.fastvit.load_state_dict(checkpoint['state_dict'])
        self.fastvit.eval()
        #self.fastvit_inf=reparameterize_model(self.fastvit)
    def forward(self,x):
        with torch.no_grad():
            features=self.fastvit(x)
        return features
class CombinedModel(nn.Module):
    def __init__(self,embedding_size,hidden_layer,output_layer):
        super(CombinedModel,self).__init__()
        self.fastvitmodel=FastViTModel()
        self.fc1=nn.Linear(embedding_size+1,hidden_layer)
        self.fc2=nn.Linear(hidden_layer,output_layer)
    def forward(self,image,ear):
        img_ten=self.fastvitmodel(image)
        ear = ear.view(-1, 1)
        combined_ten=torch.cat([img_ten,ear],dim=1)
        x=torch.relu(self.fc1(combined_ten))
        output=self.fc2(x)
        return output


        



In [45]:
embedd=1000
hidden=512
output_feats=1
model=CombinedModel(embedd,hidden,output_feats)

In [46]:
model.train()

CombinedModel(
  (fastvitmodel): FastViTModel(
    (fastvit): FastVit(
      (stem): Sequential(
        (0): MobileOneBlock(
          (se): Identity()
          (conv_kxk): ModuleList(
            (0): ConvNormAct(
              (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (bn): BatchNormAct2d(
                48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
                (drop): Identity()
                (act): Identity()
              )
            )
          )
          (conv_scale): ConvNormAct(
            (conv): Conv2d(3, 48, kernel_size=(1, 1), stride=(2, 2), bias=False)
            (bn): BatchNormAct2d(
              48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): Identity()
            )
          )
          (act): GELU(approximate='none')
        )
        (1): MobileOneBlock(
          (se): Identity()
          (conv_kxk): 

In [20]:
import torch
fastvit_model = FastViTModel()

# Create a dummy input
dummy_input = torch.randn(1, 3, 227, 227)  # Batch size of 1, 3 channels, 384x384 resolution

# Get the output features
output_features = fastvit_model(dummy_input)

# Determine the embedding size
embedding_size = output_features.shape[1]
print(f"Embedding Size: {embedding_size}")
print(output_features.shape)
print(output_features)

Embedding Size: 1000
torch.Size([1, 1000])
tensor([[ 1.0333e-03,  5.8794e-04,  5.7143e-05,  1.3621e-03,  2.9528e-03,
         -1.6859e-03,  1.1498e-03, -3.0116e-03, -1.3493e-03,  7.0720e-04,
         -6.6050e-04,  2.4320e-03,  1.3663e-04, -2.5631e-03,  4.7526e-04,
          1.2160e-03, -2.5054e-05, -2.6778e-04, -8.2377e-05, -8.9304e-04,
          7.7570e-04,  6.4878e-04,  2.3683e-03,  3.0630e-05,  1.8544e-03,
         -1.6451e-03, -1.1481e-03,  1.6083e-03, -9.1726e-04,  1.0645e-03,
         -1.2894e-03,  2.5256e-04, -2.1362e-04, -1.7920e-03, -3.0094e-04,
         -4.5761e-03,  1.0606e-03, -1.2428e-03,  2.6705e-04,  2.6204e-03,
          4.2598e-04, -4.0152e-04, -1.2626e-03, -1.3176e-03,  8.1993e-04,
         -4.4475e-04, -1.4953e-03, -1.2518e-03, -8.5648e-05, -3.2389e-04,
          1.0502e-03, -2.0557e-03, -1.3360e-03,  1.4830e-03,  1.0095e-03,
         -1.0444e-03, -3.2451e-03, -1.5558e-03, -1.4152e-03,  1.9347e-03,
         -9.5210e-04,  7.2415e-04, -1.5609e-04, -6.2154e-04,  1.5060e

In [21]:
image_input = torch.randn(1, 3,227, 227)  # Batch size of 1, image input
ear_value = torch.tensor([[0.3]])  # Batch size of 1, EAR value
labels = torch.tensor([1])  # Batch size of 1, labels (e.g., 1 for drowsy)

# Forward pass
output = model(image_input, ear_value)
print(output)

tensor([[0.0031]], grad_fn=<AddmmBackward0>)


In [50]:
p=next(iter(val_dl))
image_input=p['pixel_values'][2:4]
ear_value=p['avg_ear'][2:4]
print(image_input.shape,ear_value.shape)
output = model(image_input, ear_value)
print(p['labels'][2:4])
print(output)

/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/r0176.png
0.3789290488757665
0
/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/p0188.png
0.1830933966194066
0
/home/ameenudeenp/charu/ViT/DDD/Drowsy/X0961.png
0.1512244116849598
1
/home/ameenudeenp/charu/ViT/DDD/Drowsy/L0581.png
0.2453346775651516
1
/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/b0476.png
0.2405982283917033
0
/home/ameenudeenp/charu/ViT/DDD/Drowsy/J0472.png
0.1782976239981946
1
/home/ameenudeenp/charu/ViT/DDD/Drowsy/Q0499.png
0.2348038765751129
1
/home/ameenudeenp/charu/ViT/DDD/Drowsy/K0174.png
0.3507259989623773
1
torch.Size([2, 3, 256, 256]) torch.Size([2])
tensor([1, 1])
tensor([[0.0549],
        [0.0451]], grad_fn=<AddmmBackward0>)


In [20]:
p=next(iter(val_dl))
p['pixel_values'][0].shape

/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/r0176.png
0.3789290488757665
0
/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/p0188.png
0.1830933966194066
0
/home/ameenudeenp/charu/ViT/DDD/Drowsy/X0961.png
0.1512244116849598
1
/home/ameenudeenp/charu/ViT/DDD/Drowsy/L0581.png
0.2453346775651516
1
/home/ameenudeenp/charu/ViT/DDD/Non Drowsy/b0476.png
0.2405982283917033
0
/home/ameenudeenp/charu/ViT/DDD/Drowsy/J0472.png
0.1782976239981946
1
/home/ameenudeenp/charu/ViT/DDD/Drowsy/Q0499.png
0.2348038765751129
1
/home/ameenudeenp/charu/ViT/DDD/Drowsy/K0174.png
0.3507259989623773
1


torch.Size([3, 256, 256])